# Adapters: Bridging External Molecular Toolkits

MolPy is designed to interoperate—not isolate.
The adapter module provides clean, bidirectional bridges between MolPy data structures and external molecular libraries, allowing you to use powerful third-party functionality while keeping MolPy as your central data model.

At the heart of the adapters sits the Wrapper abstraction.
A Wrapper is a lightweight layer that attaches extra semantic or procedural information onto a MolPy data structure (e.g., an Atomistic), without altering the underlying molecular graph.
This lets MolPy:

* record external provenance (e.g., “generated via RDKit”)
* store mapping tables between MolPy atoms and foreign library atoms
* augment a structure with additional metadata or derived properties
* maintain stable round-trip conversions

In other words, Wrapper makes MolPy chemically interoperable.

In [ ]:
from rdkit import Chem
from molpy.adapter import RDKitWrapper, atomistic_to_mol, mol_to_atomistic

# 1. Create a molecule from SMILES (ethanol)
mol = Chem.MolFromSmiles("CCO")

# 2. Convert to MolPy Atomistic (2D structure, no coordinates yet)
atomistic = mol_to_atomistic(mol)
print(f"Converted molecule: {len(list(atomistic.atoms))} atoms")

# 3. Create wrapper for bidirectional operations
wrapper = RDKitWrapper.from_mol(mol)

# 4. Generate 3D coordinates and optimize geometry
wrapper.generate_3d(optimize=True)

# 5. Access the 3D MolPy structure
wrapper.sync()
atomistic_3d = wrapper.inner
print(f"Generated 3D structure with coordinates")
print(f"Atoms: {len(list(atomistic_3d.atoms))}")
print(f"Bonds: {len(list(atomistic_3d.bonds))}")


### How 3D Generation Works

`wrapper.generate_3d(optimize=True)` performs these steps:

1. **Conformer sampling** – Generates initial 3D geometry using ETKDG algorithm
2. **Force field optimization** – Refines geometry using MMFF or UFF
3. **Coordinate assignment** – Updates the `Atomistic` structure with 3D coordinates

**Parameter choices:**
- `optimize=True` – Recommended for physically reasonable geometries
- `optimize=False` – Faster, but coordinates may have high energy

---

## Converting Back to RDKit

Once you've manipulated a structure in MolPy (e.g., polymer building, reactions), you can convert back to RDKit for further analysis:


In [ ]:
# Convert Atomistic back to RDKit Mol
mol_from_atomistic = atomistic_to_mol(atomistic_3d)
print(f"RDKit molecule: {mol_from_atomistic.GetNumAtoms()} atoms")

# Generate 2D drawing (SVG format)
svg = wrapper.draw(show=False)
print("Generated 2D molecular drawing (SVG)")